In [3]:
import stanza
import pandas as pd
from typing import List, Tuple
import json

In [4]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence.")
displacy.serve(doc, style="dep")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [5]:
def load_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
        return data

In [6]:
stanza.download('uk')

2020-06-12 20:44:34 INFO: Downloading default packages for language: uk (Ukrainian)...
2020-06-12 20:44:35 INFO: File exists: /home/dbabenko/stanza_resources/uk/default.zip.
2020-06-12 20:44:37 INFO: Finished downloading models and saved to /home/dbabenko/stanza_resources.


In [7]:
dataset_df = pd.read_csv('dataset/collocation-sentence.csv')

In [8]:
for i in range(0, 1000):
    print(dataset_df['collocation'].values[i], "====>", dataset_df['sentence'].values[i] )

хороший готель ====> Хороший готель.
полную терасовані футболку ====> Повернули полную терасовані футболку.
комфортні номери ====> Комфортні номери, хороша кухня)
хороша кухня ====> Комфортні номери, хороша кухня)
прогулянка до пляжу займала ====> Прогулянка до пляжу займала лише 20 хвилин, через мальовничий парк...Наступні наші поїздки будемо намагатись планувати з проживанням в цьому готелі.
мальовничий парк ====> Прогулянка до пляжу займала лише 20 хвилин, через мальовничий парк...Наступні наші поїздки будемо намагатись планувати з проживанням в цьому готелі.
головному корпусі ====> У головному корпусі до 23ч галаслива музика і гуляння.
добре місцезнаходження ====> Добре місцезнаходження, своя приватна автостоянка.
приватна автостоянка ====> Добре місцезнаходження, своя приватна автостоянка.
смачний сніданок ====> Смачний сніданок, хороший номер, гостинний персонал.
хороший номер ====> Смачний сніданок, хороший номер, гостинний персонал.
гостинний персонал ====> Смачний сніданок, хо

In [15]:
dataset_df.head(1000)

,collocation,sentence
0,хороший готель,Хороший готель.
1,полную терасовані футболку,Повернули полную терасовані футболку.
2,комфортні номери,"Комфортні номери, хороша кухня)"
3,хороша кухня,"Комфортні номери, хороша кухня)"
4,прогулянка до пляжу займала,"Прогулянка до пляжу займала лише 20 хвилин, че..."
...,...,...
995,відмінний готель,"Відмінний готель, дуже оригінальна, яскрава, в..."
996,"дуже оригінальна, яскрава, великі просторі","Відмінний готель, дуже оригінальна, яскрава, в..."
997,"дуже чисті, приємний персонал","Відмінний готель, дуже оригінальна, яскрава, в..."
998,світлий інтер,Світлий інтер & #39; єр розташовує до нормальн...


In [18]:
todo: [
    
    "Красиві і чисті номери, чудові сніданки, поруч ринок",
    
    
    
    "Персонал дуже приємний."
    "гарне місце розташування"
    
    
    
    
    
    "Персонал був не завжди ввічливим, а так в цілому все сподобалось.",
    "В готелі персонал був не завжди дуже ввічливим, а так в цілому все сподобалось."
]

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1-"

In [10]:
nlp = stanza.Pipeline('uk')

2020-06-12 13:27:38 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-06-12 13:27:38 INFO: Use device: gpu
2020-06-12 13:27:38 INFO: Loading: tokenize


RuntimeError: CUDA error: out of memory

In [ ]:
nlp_en = stanza.Pipeline("en")

In [ ]:
# train_df = pd.read_csv("/home/dbabenko/DS/NLP/Detect-emotion-sentimental/dataset/booking/booking-train.csv")

In [ ]:
# train_df.head()

### Ananlyze collocations with noun

In [ ]:
# class TokenNode:
#     def __init__(self, token_info, head=None):
#         self.token_info = token_info
#         self.head = head
#         self.children = []
        
#     def get_pos(self):
#         return self.token_info['upos']
        
#     def get_deprel(self):
#         return self.token_info['deprel']
    
#     def get_head_id(self):
#         return self.token_info['head']
    
#     def get_id(self):
#         return int(self.token_info['id'])
    
#     def get_text(self):
#         return self.token_info['text']
    
#     def get_feats(self):
#         return self.token_info['feats']

In [ ]:
# class NounCollocationExtractor:
#     SUPPORTED_NOUN_CHILD_DEPREL = ['amod', 'advmod', 'nmod']
#     IGNORE_NOUN_CHILD_DEPREL = ['nummod:gov']
#     SUPPORTED_NOUN_HEAD_DEPREL = ['nsubj']

#     SUPPORTED_HEAD_POS = ['ADJ', 'VERB']

#     def __init__(self, nlp):
#         self.nlp = nlp

#     def extract(self, text: str):
#         doc = self.nlp(text)

#         result = []
#         for sent in doc.sentences:
#             result += self.__extract_from_sent(sent.to_dict())

#         return result

#     def __extract_from_sent(self, sent_info):
#         result = []
#         noun_ids = self.__get_all_noun_ids(sent_info)
#         if len(noun_ids) == 0:
#             return result

#         token_graph = self.__build_token_graph(sent_info)

#         for noun_id in noun_ids:
#             noun_token = token_graph[noun_id]
#             collocation = self.__extract_collocation_for_noun(noun_token)
#             if len(collocation) > 0:
#                 result.append(collocation)

#         return result

#     def __extract_collocation_for_noun(self, noun_token):
#         queue = self.__get_all_suitable_children(noun_token)
#         if self.__is_head_suitable(noun_token):
#             queue.append(noun_token.head)
#         phrase_tokens = [noun_token]

#         while queue:
#             item = queue.pop(0)
#             phrase_tokens.append(item)

#             children = item.children

#             for child in children:
#                 deprel = child.get_deprel()
#                 if deprel in self.IGNORE_NOUN_CHILD_DEPREL:
#                     return []
                
#                 if child.get_deprel() == 'advmod':
#                     queue.append(child)
                    
#                 if child.get_deprel() == 'conj':
#                     queue.append(child)


#         return self.__validate_noun_collocation(phrase_tokens)

#     def __get_all_suitable_children(self, noun_token):
#         queue = []

#         children = noun_token.children

#         for child in children:
#             deprel = child.get_deprel()
#             if deprel in self.SUPPORTED_NOUN_CHILD_DEPREL:
#                 queue.append(child)

#         return queue

#     def __is_head_suitable(self, noun_token):
#         if noun_token.head is None:
#             return False

#         if noun_token.token_info['deprel'] not in self.SUPPORTED_NOUN_HEAD_DEPREL:
#             return False

#         if noun_token.head.token_info['upos'] not in self.SUPPORTED_HEAD_POS:
#             return False

#         return True

#     def __build_token_graph(self, sent_info):
#         token_node_dict = dict()
#         for token_info in sent_info:
#             token_id = int(token_info['id'])
#             head_id = token_info['head']

#             token_node_dict[token_id] = TokenNode(token_info)

#         for token_id in token_node_dict:
#             token_node = token_node_dict[token_id]

#             head_id = token_node.get_head_id()

#             if head_id > 0:
#                 token_node.head = token_node_dict[head_id]
#                 token_node_dict[head_id].children.append(token_node)

#         return token_node_dict

#     def __get_all_noun_ids(self, sent_info):
#         ids = []
#         for token_info in sent_info:
#             if token_info['upos'] == 'NOUN':
#                 ids.append(int(token_info['id']))

#         return ids

#     def __validate_noun_collocation(self, collocation_tokens):
#         result = []
        
#         collocation_tokens.sort(key=lambda token: token.get_id())
        
#         collocation_tokens = self.__find_max_len_non_sequence_words(collocation_tokens)
        
#         if len(collocation_tokens) < 2:
#             return []
        
#         if self.__is_collocation_valid_by_pos(collocation_tokens):
#             return [item.get_text().lower() for item in collocation_tokens]
        

#         return []
    
    
#     def __find_max_len_non_sequence_words(self, collocation_tokens):
#         result = []
#         if len(collocation_tokens) == 0:
#             return result
        
#         cur_seq = [collocation_tokens[0]]
#         max_seq = cur_seq
#         for i in range(1, len(collocation_tokens)):
#             if collocation_tokens[i].get_id() - collocation_tokens[i - 1].get_id() > 1:
#                 if len(max_seq) < len(cur_seq):
#                     max_seq = cur_seq
#                 cur_seq = []
#             else:
#                 cur_seq.append(collocation_tokens[i])
                
#         return max_seq
    
    
#     def __is_collocation_valid_by_pos(self, collocation_tokens):
#         noun_token, adj_token, verb_token = None, None, None
#         for token in collocation_tokens:
#             pos = token.get_pos()
#             if pos == 'ADJ':
#                 adj_token = token
#             elif pos == 'VERB':
#                 verb_token = token
#             elif pos == 'NOUN':
#                 noun_token = token
        
#         if noun_token is None:
#             return False
        
#         if verb_token is None and adj_token is None:
#             return False
        
        
#         if verb_token is None:
#             if 'Case=Gen' in adj_token.get_feats():  # to avoid collocation like "гарячої води", but support "немає гарячої води":
#                 return False
#         else:
#             if noun_token.get_id() < verb_token.get_id(): #to avoid collocation like "колеги знайшли"
#                 return False
            
#         return True

### Collocation example

In [6]:
from collocations.noun_collocation_extractor import NounCollocationExtractor

In [7]:
extractor = NounCollocationExtractor(nlp)

In [8]:
text_example = "Не дуже привітний персонал, тому було некомфортно тут зупинятись."

In [9]:
extractor.extract(text_example)

[[{'id': '1',
   'text': 'Не',
   'lemma': 'не',
   'upos': 'PART',
   'xpos': 'Q',
   'feats': 'Polarity=Neg',
   'head': 2,
   'deprel': 'advmod',
   'misc': 'start_char=0|end_char=2'},
  {'id': '2',
   'text': 'дуже',
   'lemma': 'дуже',
   'upos': 'ADV',
   'xpos': 'Rp',
   'feats': 'Degree=Pos',
   'head': 3,
   'deprel': 'advmod',
   'misc': 'start_char=3|end_char=7'},
  {'id': '3',
   'text': 'привітний',
   'lemma': 'привітний',
   'upos': 'ADJ',
   'xpos': 'Ao-msnf',
   'feats': 'Case=Nom|Gender=Masc|Number=Sing',
   'head': 4,
   'deprel': 'amod',
   'misc': 'start_char=8|end_char=17'},
  {'id': '4',
   'text': 'персонал',
   'lemma': 'персонал',
   'upos': 'NOUN',
   'xpos': 'Ncmsnn',
   'feats': 'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing',
   'head': 0,
   'deprel': 'root',
   'misc': 'start_char=18|end_char=26'}]]

In [10]:
doc = nlp(text_example)
doc.sentences[0].print_dependencies()

('Не', '2', 'advmod')
('дуже', '3', 'advmod')
('привітний', '4', 'amod')
('персонал', '0', 'root')
(',', '8', 'punct')
('тому', '8', 'advmod')
('було', '8', 'cop')
('некомфортно', '4', 'parataxis')
('тут', '10', 'advmod')
('зупинятись', '8', 'csubj')
('.', '4', 'punct')


#### short explanation

Як спрацював для даного речення алгоритм? <br>
Спочтаку знайшли іменник *персонал*. Далі до нього всих дітей (ті хто мають індекс head 4) і відповідний deprel (advmod або amod), тут це *привітний*. Для слова *привітний* також шукаються його діти з відповідним deprel, тут це *дуже*, потім для *дуже* знайшовся child *не*, для якого вже дітей немає і тому черга розгляду стала пустою. Отже маємо токени *не дуже привітний персонал*, які в сукупності проходять додаткову перевірки (так щою вони дійсно йшли по-порядку, іменник був після дієслова, якщо воно є та інші перевірки).

In [ ]:
text_example = "Гарний готель, зручне розташування. Персонал ввічливий. Номера чисті, рушники сухі й чисті. В ванній кімнаті все охайно, є по 4 невеличких мила та 4 одноразових шампуня. А санвузлі є водонагрівач, горячої води було вдосталь. (Номер був на чотирьох осіб, мешкали двоє). Є одноразові тапочки. В номері було два невеличких лед-телевізора, мініхолодильник та кондиціонер. З вікна відкривається чудовий вид на Замкову гору. Поряд є шикарний пивний ресторан Кумпель. В готелі є камера схову, якщо треба висилятися а ще багато вільного часу - можна безкоштовно обставити речі та піти по своїх справах. Бал зняв за таргана, якого колеги знайшли в себе в номері. Підкреслю, що в нас такого не було."
text_example

In [ ]:
extractor.extract(text_example)

In [ ]:
text_example = "Маленький затишний готель у гарному й тихому районі. Номер досить просторий. Ліжко зручне. Є фен та халат, шафа, настільна лампа, стіл та 2 стільця, прикроватні тумбочки та зручно розташовані розетки, мінібар. Сніданок входить до вартості. Вибір на сніданок не великий, але все смачно. Окрім маленького шведського столу можна замовити млинці, куряче філе, яєшню чи ще щось (теж входить в ціну). Недалеко від готелю, дерев'яні сходи вгору до Пейзажної алеї та Музеї історії України."
text_example

In [ ]:
extractor.extract(text_example)

TODO: think how to avoid collocation **настільна лампа**, **прикроватні тумбочки**, **деревяні сходи вгору**

In [ ]:
doc = nlp(text_example)
doc.sentences[3].print_dependencies()

In [ ]:
doc.sentences[7].print_dependencies()

In [ ]:
doc.sentences[3]

In [ ]:
text_example = "Дуже затишний, надзвичайно чистий готель. Хоча ми приїхали дуже пізно, нас чекали і привітно обслужили. Вранці ми виїзджали дуже рано в аеропорт, тому замість сніданку нам запропонували сендвічі в дорогу. А також забезпечили трансфером. Дякуємо за хороший сервіс!."
text_example

In [ ]:
extractor.extract(text_example)

In [ ]:
doc = nlp(text_example)
doc.sentences[0].print_dependencies()

Як бачимо вище колокація *читий готель* не врахувалась, із-за того що немає звязку між читий і готель, а готель вказує на затишний, що в свою чергу містить child чистий. Варто подумати як це акууратно теж врахувати.

In [ ]:
text_example = "Дуже хороший готель! Привітні люди, почуваєш себе як в дома 😍🤗."

In [ ]:
extractor.extract(text_example)

In [ ]:
text_example = "Номер був розрахований на трьох, та ліжко (розкладний диван) для третього чоловіка був просто непридатний для того, щоб на ньому спати, це був просто жах! Сніданок - це щось смішне.."

In [ ]:
extractor.extract(text_example)

In [ ]:
text_example = "Чудовий готель! Дуже привітний персонал, красивий та чистий номер. Надзвичайно смачний сніданок та можливість на місці замовити не менш смачні перекуси чи чай/каву. Красивий та затишний задній дворик-сад де чудово можна посидіти та помилуватись заходом сонця. Лише позитивні емоції від готелю."

In [ ]:
extractor.extract(text_example)

In [ ]:
extractor.extract("Гарний готель, зручне розташування розеток.")

In [ ]:
text1 = "В ванній кімнаті все охайно, є по 4 невеличких мила та 4 одноразових шампуня."

In [ ]:
extractor.extract(text1)

In [ ]:
extractor.extract("Немає одноразових тапочок.")

TODO: this collocation should be considered

In [ ]:
extractor.extract("Одноразові тапочки відсутні.")

In [ ]:
extractor.extract("Не дуже привітний персонал, тому було некомфортно тут зупинятись.")

### Collocation in booking review

In [11]:
booking_train_df = pd.read_csv('dataset/booking/booking-train.csv')

In [12]:
booking_train_df.head()

,Unnamed: 0,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4


In [13]:
pos_texts = booking_train_df['pos_text'].values

In [14]:
neg_texts = booking_train_df['neg_text'].values

In [15]:
pos_texts

array(['Лише дівчата на рецепції - три рази мені міняли кімнату. Дякую їм.',
       'Оформлення кімнати хороше, досить приємне, на одну-дві ночі - чудовий варіант, особливо якщо врахувати ціну. Такі номери у конкурентів в два рази дорожчі.',
       "Усе відмінно, завдяки якісному сервісу ми завжди тут зупиняємося. Цього разу ресторан було заброньовано, нам люб'язно запропонували вечерю в номер, це дуже зручно. Надзвичайно люб'язний персонал. Є територія що замикається, для мешканців з авто.",
       ..., 'Зручне місце розташування, парковка, комфортний номер',
       'Рівень хорошого готелю: чисто, зручно, безкоштовні мило-шампуні, холодильник, ввічливий персонал. Все сподобалося.',
       'Великий номер, зручне ліжко, ванна кімната'], dtype=object)

In [16]:
def generate_text_from_collocaton(collocation):
    text = collocation[0]['text'].lower()
    
    for i in range(1, len(collocation)):
        token = collocation[i]
        if 'upos' in token and token['upos'] != 'PUNCT':
            text += " "
        text += token['text']
    
    return text

In [17]:
texts = neg_texts[100:200]
for text in texts:
    print("\n##################################################")
    print('text: ', text)
    collocations = extractor.extract(text)
    print("\nCollocations:\n")
    for collocation in collocations:
        print(generate_text_from_collocaton(collocation))


##################################################
text:  Непрацюючий холодильник, відсутність мікрохвильової печі у номері - при бронюванні готель заявляв наявність даного обладнання. Одноразові капці у номері відсутні, продаються по 20 гривень.

Collocations:

непрацюючий холодильник
готель заявляв наявність
одноразові капці у номері відсутні

##################################################
text:  В першому номері (потім ми попросили його змінити, й наше прохання задовільними) були поламані: злив санвузла, забитий душових злив і протікаючи шланг для душу . Раковина дуже вузенька і над нею виступає поличка дзеркала - вмитися ще той квест), на щастя у другому номері було все ОК! Додатковий збір комісії про який не попередили тут на сайті...І найголовніше це розташування, оскільки, через бюрократичні моменти, де - юре готель знаходиться не в Києві - ми годину його шукали по вулиці зазначеній тут на сайті і тільки з другого разу людина з готелю по телефону повідомила нас про розташув


Collocations:

є певні недоліки в плані чистоти
щоденного прибирання немає
радіатор ледь працює
не вистачає розеток
виникають незручності при заселенні
знайти місце реєстрації
3-у корпусі
бронювання знімають
шумоізоляція дверей жахлива

##################################################
text:  пропадало світло, правда не на довго . В ванній кімнаті були проблеми з світлом, на наше зауваження, сказали, що вже визвали електрика, але він приїде тільки після того як ми виселить Було дуже темно, приходилось завжди присвячувати телефоном . Біля одного ліжка відсутня розетка, хоча нічний стояв на тумбочці.

Collocations:

пропадало світло
ванній кімнаті
були проблеми з світлом
визвали електрика
відсутня розетка
нічний стояв

##################################################
text:  Місцезнаходження, ванна кімната

Collocations:

ванна кімната

##################################################
text:  Сніданок явно переоцінений, офіціант не вважає за потрібне принести здачу, Вайда постійно ві


Collocations:

були проблеми
безготівковим розрахунком клієнта
номери могли

##################################################
text:  Номер не відповідав заброньовані категорії, довелося годину часу сваритися на ресепшені щоб надали такий номер як був вказаний на банкінгу.

Collocations:

номер не відповідав заброньовані категорії

##################################################
text:  В туалеті пліснява на стиках плитки, брудна шторка душу, холодильник заважав спати, а коли вимкнув - закапали з нього вода, біля телевізора відірвана розетка.

Collocations:

брудна шторка душу
холодильник заважав
біля телевізора відірвана розетка

##################################################
text:  Nan

Collocations:


##################################################
text:  В цей день, коли ми приїхали, було багато людей, проводилась якась конференція, тому дуже заважали гучні сусіди в сусідньому номері, які верещали, як ненормальні, гупали, стрибали та співали пісні, тому в 3 ночі, прийшло


Collocations:

чорне волосся
не зрозумілі білі плями
дверній коробці
трохи коротка шторка в душі
вода витікала
однодольні ліжка не здригаються

##################################################
text:  Задовільна ціна як для столиці. Сподобалося що номер дуже просторий та світлий, проте дуже "втомлений", все потребує косметики. Особливо засмутив душ - з пліснявою і битою плиткою. Обрали для себе номер "комфорт", а зручності виявилися навіть гірше ніж в стандарті - дивно. А ще абсолютно не відповідає фото-опису. Дуже сильні протяги з коридору, через відсутність порогу у вхідних дверей - нонсенс. У описі вказана звукоізоляція - та де там, чутно всіх сусідів, навіть з іншого кінця коридору(знову ж таки, через двері). В цілому зосталися розчаровані, хоча територія та планування гарні. Можливо не пощастило з окремо взятим номером.

Collocations:

задовільна ціна
номер дуже просторий та світлий
особливо засмутив душ
битою плиткою
комфорт а зручності виявилися
ще абсолютно не відповідає фото


Collocations:

не приємний запах
номер не відповідає тому
там побачили мурах
раптом в когось алергія
кабельне телебачення
включивши телевізор
дали ключ
обоїх відстають
жахливому стані
двері в душі розпадаються
пульт від телевізора не працює

##################################################
text:  Далеко добиратись.

Collocations:


##################################################
text:  На жаль у вихідний грала дуже гучно музика з ресторану, що поруч до 12 години, потім ще до 2ї ночі було чути шум, перетягували столи, гриміли посудом, дівчина з рецепції нічого зробити не могла....власнику варто потурбуватися про кращу звукоізоляцію!

Collocations:

жаль у вихідний
грала дуже гучно музика з ресторану
перетягували столи
кращу звукоізоляцію

##################################################
text:  Номер 305.Замок у вхідних погано працює,вставити ключ дуже важко за 5 ти разом.
Крім цього змішувач у  умивальнику не закріплений.
 Двері на балконі виходять в стіну де в низу купа сміття.

In [18]:
text = pos_texts[4]
print(text)
extractor.extract(text)

Приїхали з сином біля сьомої ранку, сонні та втомлені, нас заселили відразу ж . Дуже вдячна . Все чисто, гарно, свіжий ремонт, подушечки . З вікна ялиночки, ніби в Карпатах живеш) )


[[{'id': '1',
   'text': 'Сонні',
   'lemma': 'соння',
   'upos': 'NOUN',
   'xpos': 'Ncmpny',
   'feats': 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur',
   'head': 0,
   'deprel': 'root',
   'misc': 'start_char=36|end_char=41'},
  {'id': '2',
   'text': 'та',
   'lemma': 'та',
   'upos': 'CCONJ',
   'xpos': 'Ccs',
   'head': 3,
   'deprel': 'cc',
   'misc': 'start_char=42|end_char=44'},
  {'id': '3',
   'text': 'втомлені',
   'lemma': 'втомлений',
   'upos': 'ADJ',
   'xpos': 'Ap--pns-ep',
   'feats': 'Aspect=Perf|Case=Nom|Number=Plur|VerbForm=Part|Voice=Pass',
   'head': 1,
   'deprel': 'conj',
   'misc': 'start_char=45|end_char=53'}],
 [{'id': '6',
   'text': 'свіжий',
   'lemma': 'свіжий',
   'upos': 'ADJ',
   'xpos': 'Afpmsnf',
   'feats': 'Case=Nom|Degree=Pos|Gender=Masc|Number=Sing',
   'head': 7,
   'deprel': 'amod',
   'misc': 'start_char=20|end_char=26'},
  {'id': '7',
   'text': 'ремонт',
   'lemma': 'ремонт',
   'upos': 'NOUN',
   'xpos': 'Ncmsnn',
   'feats': 'Animacy=In

In [20]:
doc = nlp("приємний персонал")
doc.sentences[0]

[
  {
    "id": "1",
    "text": "приємний",
    "lemma": "приємний",
    "upos": "ADJ",
    "xpos": "Afpmsnf",
    "feats": "Case=Nom|Degree=Pos|Gender=Masc|Number=Sing",
    "head": 2,
    "deprel": "amod",
    "misc": "start_char=0|end_char=8"
  },
  {
    "id": "2",
    "text": "персонал",
    "lemma": "персонал",
    "upos": "NOUN",
    "xpos": "Ncmsny",
    "feats": "Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing",
    "head": 0,
    "deprel": "root",
    "misc": "start_char=9|end_char=17"
  }
]

In [ ]:
doc.sentences[0].print_dependencies()

### Collocations in positive reviews text

In [ ]:
pos_texts = train_df['pos_text'].values[:1000]

In [ ]:
pos_collocations = []
for text in pos_texts:
    collocations = extractor.extract(text)
    if len(collocations) > 0:
        pos_collocations.append(collocations)

In [ ]:
pos_collocations

TODO: проаналізувати чому потрапляє шум. Наприклад така колокація як **кращу ціну** складно відрізнити від типу колокації роду **гарне розташування**. 

### Collocations in negative reviews text

In [ ]:
neg_texts = train_df['neg_text'].values[:1000]

In [ ]:
neg_collocations = []
for text in neg_texts:
    try:
        collocations = extractor.extract(text)
        if len(collocations) > 0:
            neg_collocations.append(collocations)
    except:
        pass

In [ ]:
neg_collocations

Як бачимо для з негативних коментрів більше шуму в колокаціях.

### Collocation for google reviews

In [ ]:
path_dir = "dataset/google/lviv"

In [ ]:
import os
files = []
for i in os.listdir(path_dir):
    if i.endswith('.json'):
        files.append(f"{path_dir}/{i}")

In [ ]:
hotel_review_dict = dict()
for file in files:
    hotel_review_dict[file] = load_json(file)

In [ ]:
topic_text_dict = dict()
count_all_reviews = 0
count_reviews = 0
for hotel, review in hotel_review_dict.items():
#     print(review.keys())
    for item in review['reviews']:
        count_all_reviews += 1
        if item['should_be_more_details'] is False:
            count_reviews += 1
            for topic, text in item['topics'].items():
                if text is not None:
                    if topic not in topic_text_dict:
                        topic_text_dict[topic] = [text]
                    else:
                        topic_text_dict[topic].append(text)


In [ ]:
topic_text_dict.keys()

In [ ]:
topic_text_dict['Breakfast']

In [ ]:
text_example = 'Зручне розташування,приємний дизайн і персонал,смачний сніданок!'

In [ ]:
extractor.extract(text_example)

In [ ]:
extractor.extract(topic_text_dict['Breakfast'][1])

In [ ]:
extractor.extract(topic_text_dict['Breakfast'][2])

In [ ]:
doc = nlp(topic_text_dict['Breakfast'][1])
doc.sentences[0].print_dependencies()

In [ ]:
text_ex = 'Зручне розташування, приємний дизайн (дуже прикольний, ласкавий) і персонал,смачний сніданок.'

In [ ]:
text_ex.split(',')

In [ ]:
doc = nlp(text_ex)
doc.sentences[0].print_dependencies()

In [ ]:
text_en = "Bill is big and honest"

In [ ]:
doc = nlp_en(text_en)
doc.sentences[0].print_dependencies()

### Topic collocation

#### Location

In [ ]:
for text in topic_text_dict['Location']:
    print("#######################################################")
    print(text)
    collocations = extractor.extract(text)
    print(collocations)

#### Breakfest collocation

In [ ]:
for text in topic_text_dict['Breakfast']:
    print("#######################################################")
    print(text)
    collocations = extractor.extract(text)
    print(collocations)

In [ ]:
text_analyze = "Дуже хороше місце для відпочинку після насиченого екскурсійного дня.приємний персонал.смачні сніданки"

In [ ]:
extractor.extract(text_analyze)

In [ ]:
doc = nlp(text_analyze)
doc.sentences[0].print_dependencies()

In [ ]:
doc.sentences[0]

In [ ]:
doc = nlp("Вцілому готель хороший, на сніданок великий вибір страв, але цього разу якась вже підгулявша кімната в мене, багато дрібниць, які потребують ремонту")
doc.sentences[0].print_dependencies()

In [ ]:
 extractor.extract("великі, чисті номери")

In [ ]:
doc.sentences[0]

In [ ]:
doc = nlp("супер розташування")
doc.sentences[0].print_dependencies()

In [ ]:
doc.sentences[0]

In [ ]:
todo_texts = [
    "Чисто, охайно, чай/кава/вода безкоштовна, непогані сніданки і 5 хв до центра міста.",
    "дуже гарний готель, зі смачними сніданками",
    "на сніданок великий вибір страв",
    "Холодна та не свіжа їжа на сніданок",
    "Як на мене ідеальне місце для сніданку.",
    "Хороший вибір їжі на сніданок.",
    " Їжа смачна, обслуговування на високому рівні.",
    "Дуже приємний сніданок, сервіс найвищого рівня.",
    " В номері було так жарко що довелось відкривати вікна))) сніданки просто супер.",
    
    "Персонал дуже привітний, обслуговування банкету було на вищому рівні"
    
]

In [ ]:
text = "Їжа смачна, обслуговування на високому рівні."

In [ ]:
doc = nlp(text)
doc.sentences[0].print_dependencies()

In [ ]:
extractor.extract(text)

#### Service collocation

In [ ]:
topic_text_dict.keys()

In [ ]:
for text in topic_text_dict['Room entertainment']:
    print("#######################################################")
    print(text)
    collocations = extractor.extract(text)
    print(collocations)

In [ ]:
text_todo = [
    "Душ з гарячою водою і підлогу з підігрівом.",
    "Номер маленький для двомісного стандарту, в душі погано зливається вода, їжа не надто.",
    "Ні телевізора, wi-fi рівень сигналу слабкий і постійно обривається коротше немає його.",
    " наприклад, батарейки в  пульті сіли, чи вода в прасці закінчилась, а праска в коридорі і брудна",
    "В конференц румі було затишно, проте заважали працювати запахи з кухні та погане кондиціонування повітря."

]

In [ ]:
extractor.extract("Телевізор в номері на стіні збоку, погана шумоізоляція, чутно сусідів.")

In [ ]:
extractor.extract("Все влаштовує, тільки білизна брудна")

In [ ]:
doc = nlp("Недорого, є кухня, туалет, душ.")
doc.sentences[0].print_dependencies()

In [ ]:
doc.sentences[0]

In [ ]:
doc.sentences[0]

In [ ]:
extractor.extract("надзвичайно смачний сніданок та можливість на місці замовити не менш смачні перекуси чи чай/каву.")

In [ ]:
import tokenize_uk

In [ ]:
tokenize_uk.tokenize_words("Привітний персонал, гарна кімната.")

### Extract collocations for hotel

In [ ]:
hotel_review_dict['dataset/google/lviv/Вітан.json']['reviews']

In [ ]:
%%time
hotel_collocation_dict = dict()
for hotel, review in hotel_review_dict.items():
    hotel_collocation_dict[hotel] = []
    for item in review['reviews']:
            if item['should_be_more_details'] is False:
#                 print(item['text'])
                hotel_collocation_dict[hotel] += extractor.extract(item['text'])
                    

In [ ]:
import numpy as np

In [ ]:
np.unique(list(hotel_collocation_dict.values()))

In [ ]:
extractor.extract("Недорого, є кухня, туалет, душ.")

In [ ]:
hotel_collocation_dict

In [ ]:
len(hotel_review_dict)

In [ ]:
for g, t in list(hotel_review_dict.items())[:1]:
    print(t)